<a href="https://colab.research.google.com/github/jaideep11061982/ObjectDetection/blob/master/spdcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Elin24/SPDCN-CAC.git

Cloning into 'SPDCN-CAC'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 63 (delta 11), reused 39 (delta 0), pack-reused 0
Unpacking objects: 100% (63/63), 280.76 KiB | 9.36 MiB/s, done.


In [ ]:
!wget https://portland-my.sharepoint.com/personal/wlin38-c_my_cityu_edu_hk/_layouts/15/onedrive.aspx?ga=1&id=%2Fpersonal%2Fwlin38%2Dc%5Fmy%5Fcityu%5Fedu%5Fhk%2FDocuments%2Fpublic%2Fbmvc%2D22%2Fckpt%5Fepoch%5Fbest%2Epth&parent=%2Fpersonal%2Fwlin38%2Dc%5Fmy%5Fcityu%5Fedu%5Fhk%2FDocuments%2Fpublic%2Fbmvc%2D22&p=14

--2023-02-22 14:07:47--  https://portland-my.sharepoint.com/personal/wlin38-c_my_cityu_edu_hk/_layouts/15/onedrive.aspx?ga=1
Resolving portland-my.sharepoint.com (portland-my.sharepoint.com)... 13.107.136.8, 13.107.138.8, 2620:1ec:8f8::8, ...
Connecting to portland-my.sharepoint.com (portland-my.sharepoint.com)|13.107.136.8|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-02-22 14:07:48 ERROR 403: Forbidden.



In [ ]:
import os 
os.sys.path.append('SPDCN-CAC')
#%cd /content/SPDCN-CAC

In [ ]:
tase

In [ ]:

#from SPDCN-CAC import *
from matplotlib import pyplot as plt
import cv2
from PIL import Image
import numpy as np
import torch
from datasets.utils import NormalSample, jpg2id

In [ ]:
boxes= [[[ 143,266  ],[ 143,317  ],[ 216,317  ],  [  216,  266  ]  ],
            [   [     297,   66  ], [   297, 86  ], [ 343,  86  ], [ 343,  66 ]  ]]
nboxes.shape

(2, 4)

In [ ]:
nboxes = np.array(boxes, dtype='float32') # ((w1, h1), (w1, h2), (w2, h2), (w2, h1)) # N 4 2
print(nboxes)
nboxes[:, :, 0] = nboxes[:, :, 0]  
nboxes[:, :, 1] = nboxes[:, :, 1]  
print(nboxes[:,:,1] ,nboxes[:,:,0])
box_lt = nboxes[:, 0, :]
##print(box_lt)
box_rb = nboxes[:, 2, :]
#print(box_rb)
nboxes = np.stack((box_lt, box_rb), axis=1)
print(nboxes.shape)
resize_boxes = nboxes.tolist()
resize_boxes[0],resize_boxes[1]

[[[143. 266.]
  [143. 317.]
  [216. 317.]
  [216. 266.]]

 [[297.  66.]
  [297.  86.]
  [343.  86.]
  [343.  66.]]]
[[266. 317. 317. 266.]
 [ 66.  86.  86.  66.]] [[143. 143. 216. 216.]
 [297. 297. 343. 343.]]
(2, 2, 2)


([[143.0, 266.0], [216.0, 317.0]], [[297.0, 66.0], [343.0, 86.0]])

In [ ]:
boxes = np.array([[   0,  817,  107, 2011],
       [ 174,  929, 1563, 2141]])
boxes[:,0]=boxes[:,0]*rw
boxes[:,2]=boxes[:,2]*rw

boxes[:,1]=boxes[:,1]*rh
boxes[:,3]=boxes[:,3]*rh

array([  0, 174])

In [ ]:
def spdcn_conversion(path,can_h=384,can_w=576):
    #imid = os.path.splitext(jpg)[0]
    imid=path.split('/').split('.')[0]
    print(imid)
    img = cv2.imread(path)
    H, W, _ = img.shape
    nH, nW = min(int(round(H / 16) * 16), can_h), min(int(round(W / 16) * 16), can_w)
    rh, rw = nH / H, nW / W
    ph, pw = (can_h - nH) // 2, (can_w - nW) // 2

    # resize image
    img = cv2.resize(img, (nW, nH), interpolation = cv2.INTER_AREA)
    canvas = np.zeros((can_h, can_w, 3), dtype='uint8')
    canvas[ph:ph+nH, pw:pw+nW, :] = img
    
    imgpath = os.path.join(root, imgdir, f'{imid}.jpg')
    cv2.imwrite(imgpath, canvas)
    #print(img.shape, H, W)

    # resize box
    boxes = np.array([[   0,  817,  107, 2011],
       [ 174,  929, 1563, 2141]])
    nboxes = np.array(boxes, dtype='float32') # ((w1, h1), (w1, h2), (w2, h2), (w2, h1)) # N 4 2
    #nboxes[:, :, 0] = nboxes[:, :, 0] * rw + pw
    #nboxes[:, :, 1] = nboxes[:, :, 1] * rh + ph
    nboxes[:,   0] = nboxes[:, :, 0] * rw + pw
    nboxes[:,  2] = nboxes[:, :,2] * rw + pw
    nboxes[:,  1] = nboxes[:, :, 1] * rh + ph
    nboxes[:,  3] = nboxes[:, :, 1] * rh + ph
    box_lt = nboxes[:, 0 ]
    box_rb = nboxes[:, 2, :]
    nboxes = np.stack((box_lt, box_rb), axis=1)
    resize_boxes = nboxes.tolist()
    
    # relocate point
    #pots = label['points'] 
    #npots = np.array(pots)
    #npots[:, 0] = npots[:, 0] * rw + pw
    #npots[:, 1] = npots[:, 1] * rh + ph
    #resize_pots = npots.tolist()
    
    # write info into label
    info[imid] = dict(
        imagepath = os.path.join(imgdir, f'{imid}.jpg'),
        points = resize_pots,
        boxes = resize_boxes,
        category = cates[imid]
    )

In [ ]:
img=Image.open('IMG_20221123_162507791.jpg')
w, h = img.size
#img=torch.tensor(np.asarray(img)).permute(2,0,1)
#dotmap = np.zeros((1, h, w), dtype=np.float32)
image = NormalSample()(img)


In [ ]:
import  torch.nn.functional as F
image=F.resize(image,)

In [ ]:
!pip install mmcv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 KB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 KB 21.9 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-1.7.1-py2.py3-none-any.whl size=930735 sha256=8f8a24f31f35730181bd63b11b472ddadfd71e50f39186cabe9c47f46a5bc41c
  Stored in directory: /root/.cache/pip/wheels/74/0c/f4/cafa17bca99a907f0ea624325aec45e905dd44884a47eae0bf
Successfully built mmcv


In [ ]:
from models import build_model

ModuleNotFoundError: ignored

In [ ]:
from models import build_model
 
model.load_state_dict(checkpoint['model'], strict=False)

output = model(images, boxes)
output = F.relu(output, inplace=True)
output = output /128

In [ ]:
import pickle as pkl
f=open('/content/image_id_boxes_dict.pkl','rb')
img_dict=pkl.load(f)

In [ ]:
img_dict['IMG_20221123_162507791'][0:2]

array([[   0,  817,  107, 2011],
       [ 174,  929, 1563, 2141]])